

**SETUP ENVIRONMENT**


In [ ]:
# @title
!git clone https://github.com/Isi-dev/Practical-RIFE
%cd /content/Practical-RIFE
!pip install git+https://github.com/rk-exxec/scikit-video.git@numpy_deprecation
!mkdir -p /content/Practical-RIFE/train_log
!wget -q https://huggingface.co/Isi99999/Frame_Interpolation_Models/resolve/main/4.25/train_log/IFNet_HDv3.py -O /content/Practical-RIFE/train_log/IFNet_HDv3.py
!wget -q https://huggingface.co/Isi99999/Frame_Interpolation_Models/resolve/main/4.25/train_log/RIFE_HDv3.py -O /content/Practical-RIFE/train_log/RIFE_HDv3.py
!wget -q https://huggingface.co/Isi99999/Frame_Interpolation_Models/resolve/main/4.25/train_log/refine.py -O /content/Practical-RIFE/train_log/refine.py
!wget -q https://huggingface.co/Isi99999/Frame_Interpolation_Models/resolve/main/4.25/train_log/flownet.pkl -O /content/Practical-RIFE/train_log/flownet.pkl
!echo "✅ All setup steps completed successfully!"


**IMAGE INTERPOLATION to GIF**

Run and Upload Two Images. Make sure both images have the same alpha channel profile. If one image is RGB and the other is RGBA, the interpolation will fail. The number of frames in the output gif file = 2 exponential(EXP). FPS is the number of Frames Per Second displayed by the output gif file.

In [ ]:
# @title
EXP = 4 # @param {"type":"number"}
FPS = 10 # @param {"type":"number"}


from google.colab import files
import os
import shutil
# Delete the existing output folder if it exists
output_dir = "/content/Practical-RIFE/output"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
from PIL import Image
uploaded = files.upload()
target_dir = "/content/Practical-RIFE"
os.makedirs(target_dir, exist_ok=True)

image_paths = []
for i, (filename, file_data) in enumerate(uploaded.items()):
    ext = os.path.splitext(filename)[1].lower()
    image_path = os.path.join(target_dir, f"{i}{ext}")

    with open(image_path, "wb") as f:
        f.write(file_data)

    image_paths.append(image_path)

image_0 = image_paths[0] if len(image_paths) > 0 else None
image_1 = image_paths[1] if len(image_paths) > 1 else None

if image_0 is not None and image_1 is not None:
    with Image.open(image_0) as img:
        width, height = img.size
    !python3 inference_img.py --img {image_0} {image_1} --exp {EXP}
    !ffmpeg -r {FPS} -f image2 -i output/img%d.png -s {width}x{height} -vf "split[s0][s1];[s0]palettegen=stats_mode=single[p];[s1][p]paletteuse=new=1" output/slomo.gif -loglevel error -y
    from IPython.display import display, Image
    display(Image(filename="output/slomo.gif"))
else:
    print("You need to select two images!")

**IMAGE INTERPOLATION to VIDEO**

Run and Upload Two Images. Make sure both images have the same alpha channel profile. If one image is RGB and the other is RGBA, the interpolation will fail. The number of frames in the output Video = 2 exponential(EXP). FPS is the number of Frames Per Second of the output video.

In [ ]:
# @title
EXP = 4 # @param {"type":"number"}
FPS = 30 # @param {"type":"number"}


from google.colab import files
import os
import shutil
# Delete the existing output folder if it exists
output_dir = "/content/Practical-RIFE/output"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
from PIL import Image
uploaded = files.upload()
target_dir = "/content/Practical-RIFE"
os.makedirs(target_dir, exist_ok=True)

image_paths = []
for i, (filename, file_data) in enumerate(uploaded.items()):
    ext = os.path.splitext(filename)[1].lower()
    image_path = os.path.join(target_dir, f"{i}{ext}")

    with open(image_path, "wb") as f:
        f.write(file_data)

    image_paths.append(image_path)

image_0 = image_paths[0] if len(image_paths) > 0 else None
image_1 = image_paths[1] if len(image_paths) > 1 else None

if image_0 is not None and image_1 is not None:
    with Image.open(image_0) as img:
        width, height = img.size
    !python3 inference_img.py --img {image_0} {image_1} --exp {EXP}
    !ffmpeg -r {FPS} -f image2 -i output/img%d.png -s {width}x{height} -vcodec libx264 -crf 23 -preset fast -pix_fmt yuv420p output/vid_from_images.mp4 -loglevel error -y
    from IPython.display import display as displayVid, Video as outVid
    displayVid(outVid("output/vid_from_images.mp4", embed=True))
else:
    print("You need to select two images!")

**VIDEO INTERPOLATION**

- Run and Upload your Video. If your video has 30 frames, then a **FRAME_MULTIPLIER** of 2 will double the number of frames to 60. If the video is 15fps, then using the default **FPS** (30) will give you a smoother output video at 30fps.The duration will remain the same. Increasing the **FPS** will reduce the duration of the video, resulting in a speedup effect. Reducing the **FPS**  will increase the duration, resulting in a slow motion effect.
- The **SCALE** parameter should be set to 0.5 for 4k videos (according to the authors of the Model).
- Enabling **INCLUDE_AUDIO** will include the audio (from the input video) in the output video and the **FPS** parameter will not be used. The **FRAME_MULTIPLIER** will determine the fps of the output video with a value of 3 turning a 10fps video to a 30fps video with no change in video duration.

In [ ]:
# @title
FRAME_MULTIPLIER = 2 # @param {"type":"number"}
FPS = 30 # @param {"type":"number"}
SCALE = 1 # @param {"type":"number"}
INCLUDE_AUDIO = False # @param {"type":"boolean"}
from google.colab import files
import os
import glob
from IPython.display import display as displayVid, Video as outVid

uploaded = files.upload()

# Define target directory
target_dir = "/content/Practical-RIFE"
os.makedirs(target_dir, exist_ok=True)

video_path = None
for filename, file_data in uploaded.items():
    ext = os.path.splitext(filename)[1].lower()
    if ext in [".mp4", ".avi", ".mov", ".mkv"]:  # Ensure it's a valid video format
        video_path = os.path.join(target_dir, f"uploaded_video{ext}")
        with open(video_path, "wb") as f:
            f.write(file_data)
        break

# Suppress ALSA errors
os.environ["XDG_RUNTIME_DIR"] = "/tmp"
os.environ["SDL_AUDIODRIVER"] = "dummy"

# Disable warnings from ffmpeg about missing audio
os.environ["PYGAME_HIDE_SUPPORT_PROMPT"] = "1"
os.environ["FFMPEG_LOGLEVEL"] = "quiet"

if INCLUDE_AUDIO:
    !python3 inference_video.py --multi={FRAME_MULTIPLIER} --video={video_path} --scale={SCALE}
else:
    !python3 inference_video.py --multi={FRAME_MULTIPLIER} --fps={FPS} --video={video_path} --scale={SCALE}


video_folder = "/content/Practical-RIFE/"

# Find the latest MP4 file
video_files = glob.glob(os.path.join(video_folder, "*.mp4"))

if video_files:
    latest_video = max(video_files, key=os.path.getctime)
    !ffmpeg -i "{latest_video}" -vcodec libx264 -crf 23 -preset fast output_converted.mp4 -loglevel error -y

    print(f"Displaying video: {latest_video}")
    displayVid(outVid("output_converted.mp4", embed=True))
    # displayVid(outVid(latest_video, embed=True))
else:
    print("❌ No video found in output/")

